In [ ]:
from cropengine.weather import GEEWeatherDataProvider

In [ ]:
import ee
import geemap

ee.Initialize()

In [ ]:
Map = geemap.Map(basemap="SATELLITE")
Map

In [ ]:
region = Map.draw_last_feature

In [ ]:
# climdata = GEEWeatherDataProvider(latitude=52.06912171608079, longitude=10.881185799808796, start_date='2020-01-01', end_date='2022-12-31', source='era5_land')
climdata_region = GEEWeatherDataProvider(
    geometry=region,
    reducer="mean",
    start_date="2020-01-01",
    end_date="2022-12-31",
    source="era5_land",
)

In [ ]:
climdata.get_data()

In [ ]:
# climdata_region.save_weather_excel('data.xlsx')

In [1]:
import requests

def fetch_soilgrids_data(lat, lon):
    """
    Fetches specific SoilGrids v2.0 data for WOFOST/PCSE.
    Mirrors the 'fetchSoilGrids' R function logic.
    """
    
    # 1. Define the Endpoint
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"

    # 2. Define the Parameters (mapping from your R snippet)
    # Note: 'wv' variables = Volumetric Water Content (Standard V2.0 properties)
    params = {
        "lat": lat,
        "lon": lon,
        "depth": ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm"],
        "property": [
            "bdod",     # Bulk Density (cg/cm3)
            "clay",     # Clay (g/kg)
            "sand",     # Sand (g/kg)
            "soc",      # Soil Organic Carbon (dg/kg)
            "wv0010",   # Water Content at 10kPa (Volume %)
            "wv0033",   # Water Content at 33kPa (Volume %) -> Field Capacity
            "wv1500"    # Water Content at 1500kPa (Volume %) -> Wilting Point
        ],
        "value": "mean" # equivalent to summary_type='mean'
    }

    # 3. Make the Request
    print(f"Fetching data for {lat}, {lon}...")
    response = requests.get(base_url, params=params)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None

    data = response.json()
    
    # 4. Parse the complex JSON structure into a simple dictionary
    # We want to average the top 3 layers (0-30cm) for the crop model
    target_depths = ["0-5cm", "5-15cm", "15-30cm"]
    parsed_data = {}

    for layer in data['properties']['layers']:
        prop_name = layer['name']
        
        # Collect values for the relevant depths
        values = [
            d['values']['mean'] 
            for d in layer['depths'] 
            if d['label'] in target_depths and d['values']['mean'] is not None
        ]
        
        if values:
            avg_value = sum(values) / len(values)
            parsed_data[prop_name] = avg_value
        else:
            parsed_data[prop_name] = None

    return parsed_data
# --- USAGE ---
# Example coordinates (Wageningen, NL)
soil_data = fetch_soilgrids_data(lat=51.99747, lon=12.16551)
print("Fetched Data (Raw Units):")
print(soil_data)

Fetching data for 51.99747, 12.16551...
Fetched Data (Raw Units):
{'bdod': 142.33333333333334, 'clay': 102.33333333333333, 'sand': 749.6666666666666, 'soc': 304.6666666666667, 'wv0010': 416.0, 'wv0033': 333.6666666666667, 'wv1500': 72.66666666666667}
